In [1]:
# ===== Celda 1: Imports y parámetros =====
import pandas as pd, numpy as np, plotly.express as px, plotly.io as pio
import unicodedata, re, os, base64

# Parámetros de negocio
DIAS_OPERADOS  = 78
VENTANA_VENTAS = 90
FACTOR_COSTO   = 0.70
PARETO_PCT     = 0.20
OBJ_30, OBJ_26 = 30, 26

# Colores/estilos (para el HTML final)
COMEX_BLUE="#00ADEF"; GREEN_MIL="#556B2F"; AMARILLO="#F1C40F"; ROJO="#E74C3C"
GRIS_TXT="#2C3E50"; BG_LIGHT="#F3F7FA"; CARD_BG="#FFFFFF"; BORDER="#E5E7EB"
LOGO_PATH = ""  # si tienes logotipo local


In [2]:
# ===== Celda 2: Helpers =====
def _norm(s: str) -> str:
    s = unicodedata.normalize('NFKD', str(s)).encode('ascii','ignore').decode('utf-8')
    s = re.sub(r'\s+', ' ', s).strip()
    s = re.sub(r'[^0-9a-zA-Z]+','_', s)
    s = re.sub(r'_{2,}','_', s).strip('_')
    return s.lower()

def normalize_cols(df: pd.DataFrame):
    m = {c:_norm(c) for c in df.columns}
    out = df.copy(); out.columns = [m[c] for c in df.columns]
    return out, m

def to_num(df, col): df[col] = pd.to_numeric(df.get(col, np.nan), errors='coerce'); return df

def find_cost_col(df_cols):
    # 1) tokens costo+invent
    for c in df_cols:
        name = c.replace('__','_')
        if 'costo' in name and 'invent' in name:
            return c
    # 2) comunes
    for c in ['costo_inventario','valor_inventario','valor','importe']:
        if c in df_cols: return c
    # 3) fallback
    for c in df_cols:
        if c.startswith('costo'): return c
    return None

def ahorro(costo_inv, costo_diario, dias_target):
    req = (costo_diario or 0) * dias_target
    return float(max(0.0, (costo_inv or 0) - req))

def semaforo(d):
    if d <= 26: return 'Verde (≤26)'
    if d <= 45: return 'Amarillo (27–45)'
    return 'Rojo (>45)'

def pareto_por_pct_costo(df_coste: pd.DataFrame, costo_col: str, pct: float):
    tmp = df_coste.copy().sort_values(costo_col, ascending=False)
    total = tmp[costo_col].sum()
    if total <= 0: return tmp.iloc[0:0]
    tmp['acum'] = tmp[costo_col].cumsum()
    tmp['pct_acum'] = tmp['acum'] / total
    sel = tmp[tmp['pct_acum'] <= pct]
    return sel if len(sel) else tmp.head(1)


In [3]:
# ===== Celda 3: Carga de archivos =====
try:
    df_ventas; df_inv
    print("Usando DataFrames ya en memoria.")
except NameError:
    from google.colab import files
    print("Sube el archivo de VENTAS (detalle)."); up_v = files.upload(); ventas_file = list(up_v.keys())[0]
    print("Sube el archivo de INVENTARIO.");       up_i = files.upload(); inv_file    = list(up_i.keys())[0]
    df_ventas = pd.read_excel(ventas_file)
    df_inv    = pd.read_excel(inv_file)

print("Ventas shape:", df_ventas.shape)
print("Inventario shape:", df_inv.shape)


Sube el archivo de VENTAS (detalle).


Saving Ventas devolucion detalle 90 Dias.xlsx to Ventas devolucion detalle 90 Dias.xlsx
Sube el archivo de INVENTARIO.


Saving Inventario al dia 20-08-2025.xlsx to Inventario al dia 20-08-2025.xlsx
Ventas shape: (21565, 31)
Inventario shape: (5213, 15)


In [4]:
# ===== Celda 4: Normalización y unificación de 'nombre_tienda' =====
df_ventas, map_v = normalize_cols(df_ventas)
df_inv,    map_i = normalize_cols(df_inv)

print("== INVENTARIO (original -> normalizada) ==")
for k,v in map_i.items(): print(f"  - {k} -> {v}")
print("\n== VENTAS (original -> normalizada) ==")
for k,v in map_v.items(): print(f"  - {k} -> {v}")

# Ventas: si no existe nombre_tienda, créalo desde nombre_de_la_tienda
if 'nombre_tienda' not in df_ventas.columns and 'nombre_de_la_tienda' in df_ventas.columns:
    df_ventas['nombre_tienda'] = df_ventas['nombre_de_la_tienda']
if 'nombre_tienda' in df_ventas.columns:
    df_ventas['nombre_tienda'] = df_ventas['nombre_tienda'].astype(str).str.strip().str.upper()

# Inventario: si no existe nombre_tienda, créalo desde nombresucursal
if 'nombre_tienda' not in df_inv.columns and 'nombresucursal' in df_inv.columns:
    df_inv['nombre_tienda'] = df_inv['nombresucursal']
if 'nombre_tienda' in df_inv.columns:
    df_inv['nombre_tienda'] = df_inv['nombre_tienda'].astype(str).str.strip().str.upper()

# Campos clave adicionales
# Ventas: subtotal y fecha
if 'subtotal' not in df_ventas.columns and 'subtotal_sin_iva' in df_ventas.columns:
    df_ventas['subtotal'] = df_ventas['subtotal_sin_iva']
df_ventas['fecha_negocio'] = pd.to_datetime(
    df_ventas['fecha_negocio'] if 'fecha_negocio' in df_ventas.columns else df_ventas.get('fecha_de_negocio'),
    dayfirst=True, errors='coerce'
)
df_ventas = to_num(df_ventas, 'subtotal')

# Inventario: inventario
if 'inventario' not in df_inv.columns:
    for alt in ['existencias','stock','cantidad']:
        if alt in df_inv.columns:
            df_inv['inventario'] = df_inv[alt]; break
df_inv = to_num(df_inv, 'inventario')

# Excluir devoluciones si viene 'tipo'
if 'tipo' in df_ventas.columns:
    df_ventas = df_ventas[~df_ventas['tipo'].astype(str).str.contains('devol', case=False, na=False)].copy()

print("\nColumnas clave presentes:")
print("VENTAS:", [c for c in ['nombre_tienda','subtotal','fecha_negocio'] if c in df_ventas.columns])
print("INVENTARIO:", [c for c in ['nombre_tienda','inventario'] if c in df_inv.columns])


== INVENTARIO (original -> normalizada) ==
  - No -> no
  - Grupo -> grupo
  - Tienda -> tienda
  - NombreSucursal -> nombresucursal
  - Articulo -> articulo
  - Descripcion -> descripcion
  - Localidad -> localidad
  - Estado -> estado
  - Inventario -> inventario
  - FechaCreacion -> fechacreacion
  - FechaActualizacion -> fechaactualizacion
  - FechaReplicacion -> fechareplicacion
  - CostoPromedio -> costopromedio
  - Costo Inventario -> costo_inventario
  - Porcentaje del total -> porcentaje_del_total

== VENTAS (original -> normalizada) ==
  - No -> no
  - Tipo -> tipo
  - Grupo -> grupo
  - Tienda -> tienda
  - Nombre de la tienda -> nombre_de_la_tienda
  - Registradora -> registradora
  - Fecha de negocio -> fecha_de_negocio
  - Transacción -> transaccion
  - Línea de transacción -> linea_de_transaccion
  - Línea del producto -> linea_del_producto
  - Artículo -> articulo
  - Descripción -> descripcion
  - Cantidad -> cantidad
  - LocModSeq -> locmodseq
  - Localidad -> localid

In [5]:
# ===== Celda 5: Detectar costo y ALIAS 'costo_inventario' =====
warn_msgs = []
COSTO_COL = find_cost_col(df_inv.columns)

if COSTO_COL is None:
    cand_cprom = [c for c in df_inv.columns if ('costo' in c and 'prom' in c)] + ['costo_promedio','costopromedio','costo_unitario']
    CPRO = next((c for c in cand_cprom if c in df_inv.columns), None)
    if CPRO is not None and 'inventario' in df_inv.columns:
        df_inv[CPRO] = pd.to_numeric(df_inv[CPRO], errors='coerce')
        df_inv['inventario'] = pd.to_numeric(df_inv['inventario'], errors='coerce')
        df_inv['costo_inventario'] = (df_inv['inventario'] * df_inv[CPRO]).fillna(0)
        COSTO_COL = 'costo_inventario'
        warn_msgs.append(f"Se calculó costo_inventario = inventario * {CPRO}.")
    else:
        df_inv['costo_inventario'] = 0.0
        COSTO_COL = 'costo_inventario'
        warn_msgs.append("No se encontró columna de costo ni costo_promedio; se asignó 0.")
else:
    df_inv[COSTO_COL] = pd.to_numeric(df_inv[COSTO_COL], errors='coerce').fillna(0)

# === ALIAS duro/obligatorio ===
if COSTO_COL != 'costo_inventario':
    df_inv['costo_inventario'] = pd.to_numeric(df_inv[COSTO_COL], errors='coerce').fillna(0)
else:
    df_inv['costo_inventario'] = pd.to_numeric(df_inv['costo_inventario'], errors='coerce').fillna(0)

print("COSTO_COL detectada:", COSTO_COL)
print("'costo_inventario' existe?:", 'costo_inventario' in df_inv.columns)
print("Inventario filas:", len(df_inv))
print("Suma costo_inventario:", float(df_inv['costo_inventario'].sum()))


COSTO_COL detectada: costo_inventario
'costo_inventario' existe?: True
Inventario filas: 5213
Suma costo_inventario: 7167625.549427


In [6]:
# ===== Celda 6: Ventana de 90 días y KPIs globales =====
end = df_ventas['fecha_negocio'].max()
start = end - pd.Timedelta(days=VENTANA_VENTAS-1)
df90 = df_ventas[(df_ventas['fecha_negocio']>=start) & (df_ventas['fecha_negocio']<=end)].copy()

ventas_90 = df90['subtotal'].sum()
costo_diario_global = (ventas_90 * FACTOR_COSTO) / max(DIAS_OPERADOS,1)
costo_inv_global  = df_inv['costo_inventario'].sum()
dias_global_pesos = (costo_inv_global / costo_diario_global) if costo_diario_global>0 else np.inf
ahorro_30_global  = ahorro(costo_inv_global, costo_diario_global, OBJ_30)
ahorro_26_global  = ahorro(costo_inv_global, costo_diario_global, OBJ_26)

print("Rango:", start.date(), "→", end.date())
print("Ventas 90d:", round(ventas_90,2))
print("Costo diario global:", round(costo_diario_global,2))
print("Costo inv global:", round(costo_inv_global,2))
print("Días inv (pesos):", round(dias_global_pesos,2))
print("Ahorro 30d:", round(ahorro_30_global,2), "| Ahorro 26d:", round(ahorro_26_global,2))


Rango: 2025-05-24 → 2025-08-21
Ventas 90d: 19366294.93
Costo diario global: 173800.08
Costo inv global: 7167625.55
Días inv (pesos): 41.24
Ahorro 30d: 1953623.07 | Ahorro 26d: 2648823.4


In [7]:
# ===== Celda 7: Días por tienda =====
venta_90_tienda = df90.groupby('nombre_tienda', as_index=False)['subtotal'].sum().rename(columns={'subtotal':'venta_90'})
venta_90_tienda['costo_diario'] = (venta_90_tienda['venta_90'] * FACTOR_COSTO) / max(DIAS_OPERADOS,1)

# OJO: AQUÍ NO USAR to_frame() (ya es DataFrame)
costo_inv_tienda = df_inv.groupby('nombre_tienda', as_index=False)['costo_inventario'].sum()

dias_tienda = costo_inv_tienda.merge(venta_90_tienda[['nombre_tienda','costo_diario']], on='nombre_tienda', how='left')
dias_tienda['costo_diario'] = dias_tienda['costo_diario'].fillna(0)
dias_tienda['dias_pesos']   = np.where(dias_tienda['costo_diario']>0, dias_tienda['costo_inventario']/dias_tienda['costo_diario'], np.inf)
dias_tienda['semaforo']     = dias_tienda['dias_pesos'].apply(semaforo)
dias_tienda['ahorro_30']    = dias_tienda.apply(lambda r: ahorro(r['costo_inventario'], r['costo_diario'], OBJ_30), axis=1)
dias_tienda['ahorro_26']    = dias_tienda.apply(lambda r: ahorro(r['costo_inventario'], r['costo_diario'], OBJ_26), axis=1)

print(dias_tienda.head())


            nombre_tienda  costo_inventario  costo_diario  dias_pesos  \
0              BASE AEREA      7.560087e+05  20822.786295   36.306799   
1  LA VENTA DEL ASTILLERO      5.272279e+05   8691.076064   60.663137   
2                NEXTIPAC      4.905634e+05  17652.487103   27.790045   
3        PLAZA SAN ISIDRO      1.837446e+06  45332.272821   40.532853   
4         TECHNOLOGY PARK      6.913961e+05  13618.521269   50.768806   

           semaforo      ahorro_30      ahorro_26  
0  Amarillo (27–45)  131325.117389  214616.262568  
1        Rojo (>45)  266495.654237  301259.958493  
2  Amarillo (27–45)       0.000000   31598.741689  
3  Amarillo (27–45)  477478.169583  658807.260865  
4        Rojo (>45)  282840.422773  337314.507850  


In [8]:
print(df_inv.columns.tolist())


['no', 'grupo', 'tienda', 'nombresucursal', 'articulo', 'descripcion', 'localidad', 'estado', 'inventario', 'fechacreacion', 'fechaactualizacion', 'fechareplicacion', 'costopromedio', 'costo_inventario', 'porcentaje_del_total', 'nombre_tienda']


In [9]:
# ===== Celda 7.0: Utilidad robusta + parche en df_inv =====

def asegurar_col_costo(df, prefer='costo_inventario'):
    """
    Garantiza que el DF tenga una columna de costo de inventario y devuelve:
    (df, nombre_columna_costo_que_si_existe)
    Reglas:
      1) Si 'prefer' está, usarla (numérica).
      2) Sino, buscar cualquier col con 'costo' y 'invent' en el nombre.
      3) Sino, intentar inventario * costo_promedio/costopromedio/costo_unitario.
      4) Sino, crear 'prefer' en 0.
    Además, si el nombre detectado != 'prefer', crea un alias 'prefer' espejeado.
    """
    cols = list(df.columns)

    # 1) preferida
    if prefer in df.columns:
        df[prefer] = pd.to_numeric(df[prefer], errors='coerce').fillna(0)
        costo_col = prefer
    else:
        # 2) variante compatible costo+invent
        cand = [c for c in cols if ('costo' in c and 'invent' in c)]
        if cand:
            costo_col = cand[0]
            df[costo_col] = pd.to_numeric(df[costo_col], errors='coerce').fillna(0)
        else:
            # 3) calcular inventario * costo_promedio / costopromedio / costo_unitario
            inv_col = 'inventario' if 'inventario' in df.columns else None
            cprom = next((c for c in cols if ('costo' in c and 'prom' in c) or c in ['costo_promedio','costopromedio','costo_unitario']), None)
            if inv_col and cprom:
                df[inv_col] = pd.to_numeric(df[inv_col], errors='coerce')
                df[cprom]   = pd.to_numeric(df[cprom], errors='coerce')
                df[prefer]  = (df[inv_col]*df[cprom]).fillna(0)
                costo_col   = prefer
            else:
                # 4) fallback
                df[prefer] = 0.0
                costo_col  = prefer

    # Crear/actualizar alias 'prefer' si el nombre real no es el preferido
    if costo_col != prefer:
        df[prefer] = pd.to_numeric(df[costo_col], errors='coerce').fillna(0)

    # Asegurar float
    df[prefer] = pd.to_numeric(df[prefer], errors='coerce').fillna(0)

    return df, prefer  # devolvemos alias garantizado

# Parchear df_inv AHORA
df_inv, COSTO_ALIAS = asegurar_col_costo(df_inv, prefer='costo_inventario')
print("COSTO_ALIAS en df_inv:", COSTO_ALIAS, "| Suma:", float(df_inv[COSTO_ALIAS].sum()))


COSTO_ALIAS en df_inv: costo_inventario | Suma: 7167625.549427


In [10]:
# ===== Celda 7.0: Reparación universal de columna de costo =====
import re, pandas as pd, numpy as np

def ensure_cost_col(DF, alias='costo_inventario'):
    """
    Garantiza que el DataFrame DF tenga la columna 'alias' con valores numéricos.
    1) Si ya existe 'alias', la tipa a numérico.
    2) Si no, busca cualquier columna que cumpla regex 'costo.*invent' (ej. 'costo_inventario').
    3) Si no, intenta DF['inventario'] * [costo_promedio|costopromedio|costo_unitario].
    4) Si no, crea 'alias' = 0.0
    Devuelve: (DF, nombre_columna_origen_usada)
    """
    cols = list(DF.columns)
    if alias in DF.columns:
        DF[alias] = pd.to_numeric(DF[alias], errors='coerce').fillna(0)
        return DF, alias

    # 2) buscar variante 'costo.*invent'
    patt = re.compile(r'costo.*invent', re.I)
    cand = [c for c in cols if patt.search(c)]
    if cand:
        DF[alias] = pd.to_numeric(DF[cand[0]], errors='coerce').fillna(0)
        return DF, cand[0]

    # 3) inventario * costo promedio/costopromedio/costo_unitario
    inv_col = 'inventario' if 'inventario' in DF.columns else None
    cpros   = [c for c in cols if ('costo' in c and 'prom' in c)] + ['costo_promedio','costopromedio','costo_unitario']
    cprom   = next((c for c in cpros if c in DF.columns), None)
    if inv_col and cprom:
        DF[inv_col] = pd.to_numeric(DF[inv_col], errors='coerce')
        DF[cprom]   = pd.to_numeric(DF[cprom], errors='coerce')
        DF[alias]   = (DF[inv_col]*DF[cprom]).fillna(0)
        return DF, f"{inv_col}*{cprom}"

    # 4) fallback
    DF[alias] = 0.0
    return DF, 'fallback_0'

def debug_cols(name, DF):
    print(f"[DEBUG] {name}: cols={list(DF.columns)[:12]}... filas={len(DF)} tiene 'costo_inventario'? {'costo_inventario' in DF.columns}")

# Asegura en df_inv AHORA (base)
df_inv, src = ensure_cost_col(df_inv, alias='costo_inventario')
print(f"[df_inv] costo_inventario OK (origen: {src}) | suma={float(df_inv['costo_inventario'].sum()):,.2f}")

# Si ya tienes otros DFs creados antes (por ejecutar celdas fuera de orden), también se curan aquí:
for name in ['costo_inv_tienda','inv_global_sku','inv_tienda_sku','pareto_global_base','pareto_tienda_base']:
    if name in globals() and isinstance(globals()[name], pd.DataFrame):
        globals()[name], src2 = ensure_cost_col(globals()[name], alias='costo_inventario')
        print(f"[{name}] asegurado (origen: {src2})")
        debug_cols(name, globals()[name])

debug_cols('df_inv', df_inv)


[df_inv] costo_inventario OK (origen: costo_inventario) | suma=7,167,625.55
[costo_inv_tienda] asegurado (origen: costo_inventario)
[DEBUG] costo_inv_tienda: cols=['nombre_tienda', 'costo_inventario']... filas=7 tiene 'costo_inventario'? True
[DEBUG] df_inv: cols=['no', 'grupo', 'tienda', 'nombresucursal', 'articulo', 'descripcion', 'localidad', 'estado', 'inventario', 'fechacreacion', 'fechaactualizacion', 'fechareplicacion']... filas=5213 tiene 'costo_inventario'? True


In [11]:
# ===== Celda 7: Días por tienda + Pareto (con alias garantizado) =====

# --- Días por tienda (EN PESOS) ---
venta_90_tienda = (
    df90.groupby('nombre_tienda', as_index=False)['subtotal']
        .sum()
        .rename(columns={'subtotal':'venta_90'})
)
venta_90_tienda['costo_diario'] = (venta_90_tienda['venta_90'] * 0.70) / 78.0

# Asegurar que df_inv tenga 'costo_inventario' (por si reejecutaste fuera de orden)
df_inv, _ = ensure_cost_col(df_inv, 'costo_inventario')

# >>> OJO: aquí antes te tronaba; ahora va con alias garantizado y SIN .to_frame()
costo_inv_tienda = (
    df_inv.groupby('nombre_tienda', as_index=False)['costo_inventario']
          .sum()
          .rename(columns={'costo_inventario':'costo_inventario'})
)

dias_tienda = costo_inv_tienda.merge(
    venta_90_tienda[['nombre_tienda','costo_diario']],
    on='nombre_tienda', how='left'
)
dias_tienda['costo_diario'] = dias_tienda['costo_diario'].fillna(0)
dias_tienda['dias_pesos']   = np.where(
    dias_tienda['costo_diario']>0,
    dias_tienda['costo_inventario']/dias_tienda['costo_diario'],
    np.inf
)

# --- Pareto global en pesos (20% del costo) ---
inv_global_sku = (
    df_inv.groupby(['articulo','descripcion'], as_index=False)['costo_inventario']
          .sum()
)
ventas_global_sku = (
    df90.groupby(['articulo','descripcion'], as_index=False)['subtotal'].sum()
)
pareto_global_base = inv_global_sku.merge(
    ventas_global_sku, on=['articulo','descripcion'], how='left'
).fillna(0)
pareto_global_base, _ = ensure_cost_col(pareto_global_base, 'costo_inventario')

tmp = pareto_global_base.sort_values('costo_inventario', ascending=False).copy()
total = tmp['costo_inventario'].sum()
tmp['acum'] = tmp['costo_inventario'].cumsum()
tmp['pct_acum'] = np.where(total>0, tmp['acum']/total, 0)
pg = tmp[tmp['pct_acum'] <= 0.20]
if pg.empty and not tmp.empty:
    pg = tmp.head(1)
pg['costo_diario'] = (pg['subtotal']*0.70)/78.0
pg['dias_pesos']   = np.where(pg['costo_diario']>0, pg['costo_inventario']/pg['costo_diario'], np.inf)

# --- Pareto por tienda (20%) ---
inv_tienda_sku = (
    df_inv.groupby(['nombre_tienda','articulo','descripcion'], as_index=False)['costo_inventario']
          .sum()
)
ventas_tienda_sku = (
    df90.groupby(['nombre_tienda','articulo','descripcion'], as_index=False)['subtotal'].sum()
)
pareto_tienda_base = inv_tienda_sku.merge(
    ventas_tienda_sku, on=['nombre_tienda','articulo','descripcion'], how='left'
).fillna(0)
pareto_tienda_base, _ = ensure_cost_col(pareto_tienda_base, 'costo_inventario')

pareto_tienda_dict = {}
for t in pareto_tienda_base['nombre_tienda'].unique():
    sub = pareto_tienda_base.loc[
        pareto_tienda_base['nombre_tienda']==t,
        ['articulo','descripcion','costo_inventario','subtotal']
    ].copy()
    sub, _ = ensure_cost_col(sub, 'costo_inventario')  # blindaje dentro del loop
    if sub.empty:
        continue
    sub = sub.sort_values('costo_inventario', ascending=False)
    total_t = sub['costo_inventario'].sum()
    sub['acum'] = sub['costo_inventario'].cumsum()
    sub['pct_acum'] = np.where(total_t>0, sub['acum']/total_t, 0)
    sel = sub[sub['pct_acum'] <= 0.20]
    if sel.empty and not sub.empty:
        sel = sub.head(1)
    sel['costo_diario'] = (sel['subtotal']*0.70)/78.0
    sel['dias_pesos']   = np.where(sel['costo_diario']>0, sel['costo_inventario']/sel['costo_diario'], np.inf)
    pareto_tienda_dict[t] = sel

print("OK: días por tienda y paretos generados.")
print("Preview días_tienda:")
print(dias_tienda.head())
print("Pareto Global filas:", len(pg), " | Tiendas en pareto:", len(pareto_tienda_dict))


OK: días por tienda y paretos generados.
Preview días_tienda:
            nombre_tienda  costo_inventario  costo_diario  dias_pesos
0              BASE AEREA      7.560087e+05  20822.786295   36.306799
1  LA VENTA DEL ASTILLERO      5.272279e+05   8691.076064   60.663137
2                NEXTIPAC      4.905634e+05  17652.487103   27.790045
3        PLAZA SAN ISIDRO      1.837446e+06  45332.272821   40.532853
4         TECHNOLOGY PARK      6.913961e+05  13618.521269   50.768806
Pareto Global filas: 22  | Tiendas en pareto: 7


/tmp/ipython-input-56658503.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pg['costo_diario'] = (pg['subtotal']*0.70)/78.0
/tmp/ipython-input-56658503.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pg['dias_pesos']   = np.where(pg['costo_diario']>0, pg['costo_inventario']/pg['costo_diario'], np.inf)
/tmp/ipython-input-56658503.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [12]:
# --- Parche previo a la gráfica: asegurar 'semaforo' y valores finitos ---
import numpy as np

def _semaforo(d):
    if d <= 26:  return 'Verde (≤26)'
    if d <= 45:  return 'Amarillo (27–45)'
    return 'Rojo (>45)'

# Si no existe 'semaforo', créala
if 'semaforo' not in dias_tienda.columns:
    if 'dias_pesos' not in dias_tienda.columns:
        raise ValueError("Falta la columna 'dias_pesos' en dias_tienda.")
    # reemplazar inf/negativos raros para que la gráfica no truene
    dias_tienda = dias_tienda.copy()
    dias_tienda['dias_pesos'] = dias_tienda['dias_pesos'].replace([np.inf, -np.inf], np.nan)
    dias_tienda['dias_pesos'] = dias_tienda['dias_pesos'].fillna(0)
    dias_tienda['semaforo']   = dias_tienda['dias_pesos'].apply(_semaforo)


In [13]:
# ===== Celda 9: Gráficas Plotly =====
import plotly.express as px
import plotly.io as pio

# Paleta ya definida antes (si no, descomenta y define rápido)
try:
    COMEX_BLUE
except NameError:
    COMEX_BLUE="#00ADEF"; GREEN_MIL="#556B2F"; AMARILLO="#F1C40F"; ROJO="#E74C3C"

# -- Días de inventario por tienda (EN PESOS) --
fig_dias_tienda = px.bar(
    dias_tienda.sort_values('dias_pesos', ascending=False),
    x='nombre_tienda', y='dias_pesos', color='semaforo',
    color_discrete_map={'Verde (≤26)':GREEN_MIL,'Amarillo (27–45)':AMARILLO,'Rojo (>45)':ROJO},
    title=f"Días de Inventario por Tienda (EN PESOS · costo diario = ventas*{FACTOR_COSTO:.2f}/{DIAS_OPERADOS})",
    text='dias_pesos'
)
fig_dias_tienda.update_traces(texttemplate='%{text:.1f}', textposition='outside', cliponaxis=False)
fig_dias_tienda.update_layout(
    yaxis_title='Días',
    xaxis_title='Tienda',
    margin=dict(l=40,r=40,t=70,b=80)
)

# -- Pareto Global (20% costo inventario) --
fig_pareto_global = px.bar(
    pg.sort_values('costo_inventario', ascending=False),
    x='descripcion', y='costo_inventario',
    title="Pareto Global (20% del costo de inventario)",
    text='costo_inventario',
    color_discrete_sequence=[COMEX_BLUE]
)
fig_pareto_global.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)
fig_pareto_global.update_layout(
    yaxis_title='Costo inventario',
    xaxis_title='SKU / Descripción',
    margin=dict(l=40,r=40,t=70,b=120)
)

# Render (opcional en notebook)
fig_dias_tienda.show()
fig_pareto_global.show()
print("Listas las figuras principales.")


Listas las figuras principales.


In [14]:
fig_dias_tienda = px.bar(
    dias_tienda.sort_values('dias_pesos', ascending=False),
    x='nombre_tienda', y='dias_pesos', color='semaforo',
    color_discrete_map={'Verde (≤26)':GREEN_MIL,'Amarillo (27–45)':AMARILLO,'Rojo (>45)':ROJO},
    title=f"Días de Inventario por Tienda (EN PESOS · costo diario = ventas*{FACTOR_COSTO:.2f}/{DIAS_OPERADOS})",
    text='dias_pesos'
)
fig_dias_tienda.update_traces(texttemplate='%{text:.1f}', textposition='outside', cliponaxis=False)
fig_dias_tienda.update_layout(yaxis_title='Días', xaxis_title='Tienda', margin=dict(l=40,r=40,t=70,b=80))


In [15]:
print(dias_tienda.columns.tolist())
print(dias_tienda[['nombre_tienda','dias_pesos','semaforo']].head())


['nombre_tienda', 'costo_inventario', 'costo_diario', 'dias_pesos', 'semaforo']
            nombre_tienda  dias_pesos          semaforo
0              BASE AEREA   36.306799  Amarillo (27–45)
1  LA VENTA DEL ASTILLERO   60.663137        Rojo (>45)
2                NEXTIPAC   27.790045  Amarillo (27–45)
3        PLAZA SAN ISIDRO   40.532853  Amarillo (27–45)
4         TECHNOLOGY PARK   50.768806        Rojo (>45)


In [16]:
# === Asegurar DÍAS por SKU en Pareto GLOBAL ===
# (ya tenías pg con costo_diario y dias_pesos, sólo limpiamos valores no finitos)
pg = pg.copy()
pg['dias_pesos'] = pg['dias_pesos'].replace([np.inf, -np.inf], np.nan).fillna(0)

# === Asegurar DÍAS por SKU en Pareto POR TIENDA ===
for t, dft in pareto_tienda_dict.items():
    df_ = dft.copy()
    if 'dias_pesos' not in df_.columns:
        df_['costo_diario'] = (df_['subtotal'] * FACTOR_COSTO) / max(DIAS_OPERADOS,1)
        df_['dias_pesos']   = np.where(df_['costo_diario']>0, df_['costo_inventario']/df_['costo_diario'], np.inf)
    df_['dias_pesos'] = df_['dias_pesos'].replace([np.inf, -np.inf], np.nan).fillna(0)
    pareto_tienda_dict[t] = df_


In [17]:
# === Pastel: distribución del costo de inventario por tienda ===
inv_tienda_val = df_inv.groupby('nombre_tienda', as_index=False)['costo_inventario'].sum()
fig_pie_inv = px.pie(
    inv_tienda_val, names='nombre_tienda', values='costo_inventario',
    title="Distribución del costo de inventario por tienda", hole=0.35
)
fig_pie_inv.update_traces(textposition='inside', textinfo='percent+label')

# === Mejora de hover en Pareto Global (muestra también días por SKU) ===
fig_pareto_global.update_traces(
    hovertemplate="<b>%{x}</b><br>Costo: $%{y:,.0f}<br>Días inventario: %{customdata[0]:,.1f}",
    customdata=np.array(pg[['dias_pesos']])
)

# Render opcional:
fig_pie_inv.show()


In [18]:
# === Días de inventario por producto (Pareto Global) ===
# Asegurar columna de días y valores finitos
pg = pg.copy()
pg['dias_pesos'] = pg['dias_pesos'].replace([np.inf, -np.inf], np.nan).fillna(0)

fig_dias_pareto_global = px.bar(
    pg.sort_values('dias_pesos', ascending=False),
    x='descripcion', y='dias_pesos',
    title="Días de inventario por producto — Pareto Global",
    text='dias_pesos', color_discrete_sequence=[AMARILLO]
)
fig_dias_pareto_global.update_traces(texttemplate='%{text:.1f}', textposition='outside', cliponaxis=False)
fig_dias_pareto_global.update_layout(
    yaxis_title='Días (en pesos)',
    xaxis_title='SKU / Descripción',
    margin=dict(l=40,r=40,t=70,b=120)
)

# === Días de inventario por producto (Pareto por TIENDA) ===
pareto_dias_tienda_html = ""
for t, dft in pareto_tienda_dict.items():
    if dft.empty:
        continue
    d = dft.copy()
    # asegurar días
    if 'dias_pesos' not in d.columns:
        d['costo_diario'] = (d['subtotal'] * FACTOR_COSTO) / max(DIAS_OPERADOS,1)
        d['dias_pesos']   = np.where(d['costo_diario']>0, d['costo_inventario']/d['costo_diario'], np.inf)
    d['dias_pesos'] = d['dias_pesos'].replace([np.inf, -np.inf], np.nan).fillna(0)

    fig_d = px.bar(
        d.sort_values('dias_pesos', ascending=False),
        x='descripcion', y='dias_pesos',
        title=f"Días de inventario por producto — Pareto {t}",
        text='dias_pesos', color_discrete_sequence=[AMARILLO]
    )
    fig_d.update_traces(texttemplate='%{text:.1f}', textposition='outside', cliponaxis=False)
    fig_d.update_layout(yaxis_title='Días (en pesos)', xaxis_title='SKU / Descripción', margin=dict(l=40,r=40,t=70,b=120))
    pareto_dias_tienda_html += pio.to_html(fig_d, include_plotlyjs=False, full_html=False)

# === Ahorro por tienda (30 y 26 días) ===
# Asegurar columnas de ahorro (deberían existir desde Celda 7)
dias_tienda = dias_tienda.copy()
for col in ['ahorro_30','ahorro_26']:
    if col not in dias_tienda.columns:
        dias_tienda[col] = np.maximum(0.0, dias_tienda['costo_inventario'] - dias_tienda['costo_diario']*(30 if col=='ahorro_30' else 26))

fig_ahorro_30 = px.bar(
    dias_tienda.sort_values('ahorro_30', ascending=False),
    x='nombre_tienda', y='ahorro_30',
    title="Ahorro potencial por tienda — Ajuste a 30 días",
    text='ahorro_30', color_discrete_sequence=[GREEN_MIL]
)
fig_ahorro_30.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)
fig_ahorro_30.update_layout(yaxis_title='Ahorro ($)', xaxis_title='Tienda', margin=dict(l=40,r=40,t=70,b=80))

fig_ahorro_26 = px.bar(
    dias_tienda.sort_values('ahorro_26', ascending=False),
    x='nombre_tienda', y='ahorro_26',
    title="Ahorro potencial por tienda — Ajuste a 26 días",
    text='ahorro_26', color_discrete_sequence=[COMEX_BLUE]
)
fig_ahorro_26.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)
fig_ahorro_26.update_layout(yaxis_title='Ahorro ($)', xaxis_title='Tienda', margin=dict(l=40,r=40,t=70,b=80))

# Render opcional en notebook
# fig_dias_pareto_global.show()
# fig_ahorro_30.show()
# fig_ahorro_26.show()

print("Listas las figuras: días Pareto (global/tienda) y ahorros (30/26).")


Listas las figuras: días Pareto (global/tienda) y ahorros (30/26).


In [19]:
# === Días de inventario por producto (Pareto Global) ===
pg_dias = pg.copy()
pg_dias['dias_pesos'] = pg_dias['dias_pesos'].replace([np.inf, -np.inf], np.nan).fillna(0)

fig_dias_pareto_global = px.bar(
    pg_dias.sort_values('dias_pesos', ascending=False),
    x='descripcion', y='dias_pesos',
    title="Días de inventario por producto — Pareto Global",
    text='dias_pesos', color_discrete_sequence=[AMARILLO]
)
fig_dias_pareto_global.update_traces(texttemplate='%{text:.1f}', textposition='outside', cliponaxis=False)
fig_dias_pareto_global.update_layout(
    yaxis_title='Días (en pesos)',
    xaxis_title='SKU / Descripción',
    margin=dict(l=40,r=40,t=70,b=120)
)

# === Días de inventario por producto (Pareto por TIENDA) ===
pareto_dias_tienda_html = ""
for t, dft in pareto_tienda_dict.items():
    if dft.empty:
        continue
    d = dft.copy()
    if 'dias_pesos' not in d.columns:
        d['costo_diario'] = (d['subtotal'] * FACTOR_COSTO) / max(DIAS_OPERADOS,1)
        d['dias_pesos']   = np.where(d['costo_diario']>0, d['costo_inventario']/d['costo_diario'], np.inf)
    d['dias_pesos'] = d['dias_pesos'].replace([np.inf, -np.inf], np.nan).fillna(0)

    fig_d = px.bar(
        d.sort_values('dias_pesos', ascending=False),
        x='descripcion', y='dias_pesos',
        title=f"Días de inventario por producto — Pareto {t}",
        text='dias_pesos', color_discrete_sequence=[AMARILLO]
    )
    fig_d.update_traces(texttemplate='%{text:.1f}', textposition='outside', cliponaxis=False)
    fig_d.update_layout(
        yaxis_title='Días (en pesos)',
        xaxis_title='SKU / Descripción',
        margin=dict(l=40,r=40,t=70,b=120)
    )
    pareto_dias_tienda_html += pio.to_html(fig_d, include_plotlyjs=False, full_html=False)

# === Ahorro por tienda (30 y 26 días) ===
dias_tienda = dias_tienda.copy()
dias_tienda['ahorro_30'] = np.maximum(0.0, dias_tienda['costo_inventario'] - dias_tienda['costo_diario']*30)
dias_tienda['ahorro_26'] = np.maximum(0.0, dias_tienda['costo_inventario'] - dias_tienda['costo_diario']*26)

fig_ahorro_30 = px.bar(
    dias_tienda.sort_values('ahorro_30', ascending=False),
    x='nombre_tienda', y='ahorro_30',
    title="Ahorro potencial por tienda — Ajuste a 30 días",
    text='ahorro_30', color_discrete_sequence=[GREEN_MIL]
)
fig_ahorro_30.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)

fig_ahorro_26 = px.bar(
    dias_tienda.sort_values('ahorro_26', ascending=False),
    x='nombre_tienda', y='ahorro_26',
    title="Ahorro potencial por tienda — Ajuste a 26 días",
    text='ahorro_26', color_discrete_sequence=[COMEX_BLUE]
)
fig_ahorro_26.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)

# HTML embebido de estas nuevas figuras
dias_pareto_global_html = pio.to_html(fig_dias_pareto_global, include_plotlyjs=False, full_html=False)
ahorro_30_html = pio.to_html(fig_ahorro_30, include_plotlyjs=False, full_html=False)
ahorro_26_html = pio.to_html(fig_ahorro_26, include_plotlyjs=False, full_html=False)


In [20]:
# === Portafolio por tienda (burbuja): Venta diaria vs Días, tamaño = Costo inventario ===
import numpy as np
import plotly.express as px

dt = dias_tienda.copy()

# asegurar columnas necesarias
if 'semaforo' not in dt.columns:
    def _sem(d):
        if d <= 26:  return 'Verde (≤26)'
        if d <= 45:  return 'Amarillo (27–45)'
        return 'Rojo (>45)'
    dt['semaforo'] = dt['dias_pesos'].apply(_sem)

# clamp para evitar tamaños extremos o inf
dt['dias_pesos'] = dt['dias_pesos'].replace([np.inf, -np.inf], np.nan).fillna(0)
dt['venta_diaria'] = dt['costo_diario']  # ya es costo diario (ventas*0.70/78)

# escala de burbuja suave (raíz) para que no se coma el gráfico
dt['size_bubble'] = np.sqrt(dt['costo_inventario'].clip(lower=0))

fig_portafolio_tiendas = px.scatter(
    dt,
    x='venta_diaria', y='dias_pesos', size='size_bubble', color='semaforo',
    hover_data={'nombre_tienda':True,'costo_inventario':':,.0f','venta_diaria':':,.0f','dias_pesos':':.1f','size_bubble':False},
    color_discrete_map={'Verde (≤26)':GREEN_MIL,'Amarillo (27–45)':AMARILLO,'Rojo (>45)':ROJO},
    title="Portafolio por Tienda — Venta diaria vs Días de inventario (tamaño = costo)",
)

fig_portafolio_tiendas.update_layout(
    xaxis_title="Venta diaria (costo) = ventas*{:.0%}/{}".format(FACTOR_COSTO, DIAS_OPERADOS),
    yaxis_title="Días de inventario (en pesos)",
    margin=dict(l=40,r=40,t=70,b=60)
)

# html embebido para dashboard
portafolio_tiendas_html = pio.to_html(fig_portafolio_tiendas, include_plotlyjs=False, full_html=False)
print("Gráfica de portafolio creada.")


Gráfica de portafolio creada.


In [21]:
import pandas as pd, numpy as np
import plotly.express as px, plotly.io as pio
import os

try: FACTOR_COSTO
except NameError: FACTOR_COSTO = 0.70
try: DIAS_OPERADOS
except NameError: DIAS_OPERADOS = 78

def kpi_card(title, value, sub=None, color="#2C3E50"):
    sub_html = f'<div class="kpi-sub">{sub}</div>' if sub else ''
    return f'''
      <div class="card">
        <div class="kpi-title">{title}</div>
        <div class="kpi-value" style="color:{color}">{value}</div>
        {sub_html}
      </div>
    '''

def to_html_if_fig(fig):
    return pio.to_html(fig, include_plotlyjs=False, full_html=False) if fig is not None else ""

BG_LIGHT="#F3F7FA"; CARD_BG="#FFFFFF"; BORDER="#E5E7EB"; GRIS_TXT="#2C3E50"
COMEX_BLUE="#00ADEF"; GREEN_MIL="#556B2F"; AMARILLO="#F1C40F"; ROJO="#E74C3C"

if 'ventas_90' not in globals():
    ventas_90 = float(df90['subtotal'].sum()) if 'df90' in globals() else 0.0
if 'costo_inv_global' not in globals():
    if 'df_inv' in globals():
        if 'costo_inventario' not in df_inv.columns:
            if 'Costo Inventario' in df_inv.columns: df_inv['costo_inventario'] = pd.to_numeric(df_inv['Costo Inventario'], errors='coerce').fillna(0)
            else: df_inv['costo_inventario'] = 0.0
        costo_inv_global = float(pd.to_numeric(df_inv['costo_inventario'], errors='coerce').fillna(0).sum())
    else:
        costo_inv_global = 0.0
costo_diario_global = (ventas_90 * FACTOR_COSTO) / max(DIAS_OPERADOS,1) if 'costo_diario_global' not in globals() else costo_diario_global
dias_global_pesos = (costo_inv_global / costo_diario_global) if costo_diario_global>0 else 0.0
ahorro_30_global = max(0.0, costo_inv_global - costo_diario_global*30)
ahorro_26_global = max(0.0, costo_inv_global - costo_diario_global*26)

kpi_html = "".join([
    kpi_card("Ventas 90d (sin IVA)", f"${ventas_90:,.0f}"),
    kpi_card("Costo Inventario", f"${costo_inv_global:,.0f}"),
    kpi_card("Días de Inventario (en pesos)", f"{dias_global_pesos:,.1f} d", sub=f"Costo diario = ventas*{FACTOR_COSTO:.2f}/{DIAS_OPERADOS}"),
    kpi_card("Ahorro si ajusto a 30 días", f"${ahorro_30_global:,.0f}", color=GREEN_MIL),
    kpi_card("Ahorro si ajusto a 26 días", f"${ahorro_26_global:,.0f}", color=GREEN_MIL),
])

fig_dias_tienda_html = ""
if 'fig_dias_tienda' in globals():
    fig_dias_tienda_html = to_html_if_fig(fig_dias_tienda)
elif 'dias_tienda' in globals():
    dt = dias_tienda.copy()
    if 'semaforo' not in dt.columns:
        def _sem(d):
            if d<=26: return 'Verde (≤26)'
            if d<=45: return 'Amarillo (27–45)'
            return 'Rojo (>45)'
        dt['dias_pesos'] = dt['dias_pesos'].replace([np.inf,-np.inf], np.nan).fillna(0)
        dt['semaforo'] = dt['dias_pesos'].apply(_sem)
    fig_dt = px.bar(
        dt.sort_values('dias_pesos', ascending=False),
        x='nombre_tienda', y='dias_pesos', color='semaforo',
        color_discrete_map={'Verde (≤26)':GREEN_MIL,'Amarillo (27–45)':AMARILLO,'Rojo (>45)':ROJO},
        title=f"Días de Inventario por Tienda (EN PESOS · ventas*{FACTOR_COSTO:.2f}/{DIAS_OPERADOS})",
        text='dias_pesos'
    )
    fig_dt.update_traces(texttemplate='%{text:.1f}', textposition='outside', cliponaxis=False)
    fig_dt.update_layout(yaxis_title='Días', xaxis_title='Tienda', margin=dict(l=40,r=40,t=70,b=80))
    fig_dias_tienda_html = to_html_if_fig(fig_dt)

fig_pareto_global_html = ""
if 'fig_pareto_global' in globals():
    fig_pareto_global_html = to_html_if_fig(fig_pareto_global)
elif 'pg' in globals():
    fg = px.bar(
        pg.sort_values('costo_inventario', ascending=False),
        x='descripcion', y='costo_inventario',
        title="Pareto Global en Pesos (20% del costo)", text='costo_inventario',
        color_discrete_sequence=[COMEX_BLUE]
    )
    fg.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)
    fg.update_layout(yaxis_title='Costo inventario', xaxis_title='SKU / Descripción', margin=dict(l=40,r=40,t=70,b=120))
    fig_pareto_global_html = to_html_if_fig(fg)

dias_pareto_global_html = globals().get('dias_pareto_global_html', "")
if not dias_pareto_global_html and 'pg' in globals():
    tmp = pg.copy()
    tmp['dias_pesos'] = tmp['dias_pesos'].replace([np.inf,-np.inf], np.nan).fillna(0)
    fd = px.bar(
        tmp.sort_values('dias_pesos', ascending=False),
        x='descripcion', y='dias_pesos',
        title="Días de Inventario — Pareto Global", text='dias_pesos',
        color_discrete_sequence=[AMARILLO]
    )
    fd.update_traces(texttemplate='%{text:.1f}', textposition='outside', cliponaxis=False)
    fd.update_layout(yaxis_title='Días (en pesos)', xaxis_title='SKU / Descripción', margin=dict(l=40,r=40,t=70,b=120))
    dias_pareto_global_html = to_html_if_fig(fd)

pie_html = globals().get('pie_html', "")
if not pie_html and 'df_inv' in globals():
    inv_tienda_val = df_inv.groupby('nombre_tienda', as_index=False)['costo_inventario'].sum() if 'costo_inventario' in df_inv.columns else pd.DataFrame(columns=['nombre_tienda','costo_inventario'])
    fp = px.pie(inv_tienda_val, names='nombre_tienda', values='costo_inventario', title="Distribución del Costo de Inventario por Tienda", hole=0.35)
    fp.update_traces(textposition='inside', textinfo='percent+label')
    pie_html = to_html_if_fig(fp)

pareto_tienda_html = globals().get('pareto_tienda_html', "")
if not pareto_tienda_html and 'pareto_tienda_dict' in globals():
    pth = ""
    for t, dft in pareto_tienda_dict.items():
        if dft is None or len(dft)==0: continue
        d = dft.sort_values('costo_inventario', ascending=False).copy()
        ft = px.bar(d, x='descripcion', y='costo_inventario', title=f"Pareto por Tienda en Pesos — {t}", text='costo_inventario', color_discrete_sequence=[GREEN_MIL])
        ft.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)
        ft.update_layout(yaxis_title='Costo inventario', xaxis_title='SKU / Descripción', margin=dict(l=40,r=40,t=60,b=120))
        pth += to_html_if_fig(ft)
    pareto_tienda_html = pth

pareto_dias_tienda_html = globals().get('pareto_dias_tienda_html', "")
if not pareto_dias_tienda_html and 'pareto_tienda_dict' in globals():
    pdt = ""
    for t, dft in pareto_tienda_dict.items():
        if dft is None or len(dft)==0: continue
        d = dft.copy()
        if 'dias_pesos' not in d.columns:
            d['costo_diario'] = (d['subtotal'] * FACTOR_COSTO) / max(DIAS_OPERADOS,1)
            d['dias_pesos']   = np.where(d['costo_diario']>0, d['costo_inventario']/d['costo_diario'], np.inf)
        d['dias_pesos'] = d['dias_pesos'].replace([np.inf,-np.inf], np.nan).fillna(0)
        fd = px.bar(d.sort_values('dias_pesos', ascending=False), x='descripcion', y='dias_pesos', title=f"Días de Inventario — Pareto {t}", text='dias_pesos', color_discrete_sequence=[AMARILLO])
        fd.update_traces(texttemplate='%{text:.1f}', textposition='outside', cliponaxis=False)
        fd.update_layout(yaxis_title='Días (en pesos)', xaxis_title='SKU / Descripción', margin=dict(l=40,r=40,t=60,b=120))
        pdt += to_html_if_fig(fd)
    pareto_dias_tienda_html = pdt

ahorro_30_html = globals().get('ahorro_30_html', "")
ahorro_26_html = globals().get('ahorro_26_html', "")
if (not ahorro_30_html or not ahorro_26_html) and 'dias_tienda' in globals():
    dt2 = dias_tienda.copy()
    if 'ahorro_30' not in dt2.columns: dt2['ahorro_30'] = np.maximum(0.0, dt2['costo_inventario'] - dt2['costo_diario']*30)
    if 'ahorro_26' not in dt2.columns: dt2['ahorro_26'] = np.maximum(0.0, dt2['costo_inventario'] - dt2['costo_diario']*26)
    f30 = px.bar(dt2.sort_values('ahorro_30', ascending=False), x='nombre_tienda', y='ahorro_30', title="Ahorro Potencial por Tienda (30 días)", text='ahorro_30', color_discrete_sequence=[GREEN_MIL])
    f30.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)
    f26 = px.bar(dt2.sort_values('ahorro_26', ascending=False), x='nombre_tienda', y='ahorro_26', title="Ahorro Potencial por Tienda (26 días)", text='ahorro_26', color_discrete_sequence=[COMEX_BLUE])
    f26.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)
    ahorro_30_html = to_html_if_fig(f30)
    ahorro_26_html = to_html_if_fig(f26)

style = f"""
<style>
body {{ font-family: Inter, Arial, sans-serif; background: {BG_LIGHT}; margin: 0; }}
.header {{ display:flex; align-items:center; justify-content:center; gap:16px; padding:24px 28px; background:{CARD_BG}; border-bottom:1px solid {BORDER}; }}
.header h1 {{ margin:0; color:{GRIS_TXT}; font-size:32px; font-weight:800; }}
.section {{ padding: 28px; }}
.kpi-wrap {{ display:flex; gap:16px; justify-content:center; flex-wrap:wrap; }}
.card {{ background:{CARD_BG}; border:1px solid {BORDER}; border-radius:14px; padding:18px 20px; min-width:260px; box-shadow:0 2px 10px rgba(0,0,0,0.04); text-align:center; }}
.kpi-title {{ font-size:13px; color:#6b7280; margin-bottom:6px; text-transform:uppercase; letter-spacing:.04em; }}
.kpi-value {{ font-size:24px; font-weight:800; color:{GRIS_TXT}; }}
.kpi-sub {{ font-size:12px; color:#7f8c8d; margin-top:4px; }}
.section h2 {{ color:{GRIS_TXT}; font-size:22px; margin:8px 0 16px 0; text-align:center; }}
.block {{ background:{CARD_BG}; border:1px solid {BORDER}; border-radius:14px; padding:16px; }}
</style>
"""

html_out = f"""
<!doctype html>
<html lang="es">
<head>
<meta charset="utf-8">
{style}
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
<title>Dashboard de Optimización de Inventario</title>
</head>
<body>

<div class="header"><h1>Dashboard de Optimización de Inventario</h1></div>

<div class="section">
  <div class="block"><div class='kpi-wrap'>
    {kpi_html}
  </div></div>
</div>

<div class="section">
  <h2>Días de Inventario por Tienda</h2>
  <div class="block">{fig_dias_tienda_html if fig_dias_tienda_html else "<div style='padding:10px;color:#6b7280;'>Sin datos para mostrar.</div>"}</div>
</div>

<div class="section">
  <h2>Pareto Global en Pesos (20% del costo)</h2>
  <div class="block">{fig_pareto_global_html if fig_pareto_global_html else "<div style='padding:10px;color:#6b7280;'>Sin datos para mostrar.</div>"}</div>
</div>

<div class="section">
  <h2>Pareto Global en Días</h2>
  <div class="block">{dias_pareto_global_html if dias_pareto_global_html else "<div style='padding:10px;color:#6b7280;'>Sin datos para mostrar.</div>"}</div>
</div>

<div class="section">
  <h2>Distribución del Costo de Inventario por Tienda</h2>
  <div class="block">{pie_html if pie_html else "<div style='padding:10px;color:#6b7280;'>Sin datos para mostrar.</div>"}</div>
</div>

<div class="section">
  <h2>Pareto por Tienda en Pesos (20% del costo)</h2>
  <div class="block">{pareto_tienda_html if pareto_tienda_html else "<div style='padding:10px;color:#6b7280;'>Sin datos para mostrar.</div>"}</div>
</div>

<div class="section">
  <h2>Pareto por Tienda en Días</h2>
  <div class="block">{pareto_dias_tienda_html if pareto_dias_tienda_html else "<div style='padding:10px;color:#6b7280;'>Sin datos para mostrar.</div>"}</div>
</div>

<div class="section">
  <h2>Ahorro Potencial por Tienda (30 días)</h2>
  <div class="block">{ahorro_30_html if ahorro_30_html else "<div style='padding:10px;color:#6b7280;'>Sin datos para mostrar.</div>"}</div>
</div>

<div class="section">
  <h2>Ahorro Potencial por Tienda (26 días)</h2>
  <div class="block">{ahorro_26_html if ahorro_26_html else "<div style='padding:10px;color:#6b7280;'>Sin datos para mostrar.</div>"}</div>
</div>

</body>
</html>
"""

with open("Dashboard_Inventario.html","w",encoding="utf-8") as f:
    f.write(html_out)

print("✅ Dashboard generado: Dashboard_Inventario.html")


✅ Dashboard generado: Dashboard_Inventario.html


In [24]:
import pandas as pd, numpy as np
import plotly.express as px, plotly.io as pio
import os

# ========= Parámetros y colores =========
try: FACTOR_COSTO
except NameError: FACTOR_COSTO = 0.70
try: DIAS_OPERADOS
except NameError: DIAS_OPERADOS = 78

BG_LIGHT="#F3F7FA"; CARD_BG="#FFFFFF"; BORDER="#E5E7EB"; GRIS_TXT="#2C3E50"
COMEX_BLUE="#00ADEF"; GREEN_MIL="#556B2F"; AMARILLO="#F1C40F"; ROJO="#E74C3C"

# ========= KPI globals (si no están listos, los calculamos mínimo viable) =========
if 'ventas_90' not in globals():
    ventas_90 = float(df90['subtotal'].sum()) if 'df90' in globals() else 0.0

if 'costo_inv_global' not in globals():
    if 'df_inv' in globals():
        if 'costo_inventario' not in df_inv.columns:
            if 'Costo Inventario' in df_inv.columns:
                df_inv['costo_inventario'] = pd.to_numeric(df_inv['Costo Inventario'], errors='coerce').fillna(0)
            else:
                df_inv['costo_inventario'] = 0.0
        costo_inv_global = float(pd.to_numeric(df_inv['costo_inventario'], errors='coerce').fillna(0).sum())
    else:
        costo_inv_global = 0.0

costo_diario_global = (ventas_90 * FACTOR_COSTO) / max(DIAS_OPERADOS,1)
dias_global_pesos = (costo_inv_global / costo_diario_global) if costo_diario_global>0 else 0.0
ahorro_30_global = max(0.0, costo_inv_global - costo_diario_global*30)
ahorro_26_global = max(0.0, costo_inv_global - costo_diario_global*26)

def kpi_card(title, value, sub=None, color=GRIS_TXT):
    sub_html = f'<div class="kpi-sub">{sub}</div>' if sub else ''
    return f'''
      <div class="card">
        <div class="kpi-title">{title}</div>
        <div class="kpi-value" style="color:{color}">{value}</div>
        {sub_html}
      </div>
    '''

kpi_html = "".join([
    kpi_card("Ventas 90d (sin IVA)", f"${ventas_90:,.0f}"),
    kpi_card("Costo Inventario", f"${costo_inv_global:,.0f}"),
    kpi_card("Días de Inventario (en pesos)", f"{dias_global_pesos:,.1f} d",
             sub=f"Costo diario = ventas*{FACTOR_COSTO:.2f}/{DIAS_OPERADOS}"),
    kpi_card("Ahorro si ajusto a 30 días", f"${ahorro_30_global:,.0f}", color=GREEN_MIL),
    kpi_card("Ahorro si ajusto a 26 días", f"${ahorro_26_global:,.0f}", color=GREEN_MIL),
])

# ========= Etiquetas superiores: inventario por tienda =========
if 'dias_tienda' not in globals():
    raise ValueError("No encuentro 'dias_tienda'. Asegúrate de ejecutar las celdas de cálculo antes.")

chips_df = dias_tienda[['nombre_tienda','costo_inventario']].copy()
chips_df['costo_inventario'] = pd.to_numeric(chips_df['costo_inventario'], errors='coerce').fillna(0)
chips_df = chips_df.sort_values('costo_inventario', ascending=False)

chips_html = ""
for _,r in chips_df.iterrows():
    chips_html += f"""
      <div class="chip">
        <div class="chip-name">{r['nombre_tienda']}</div>
        <div class="chip-value">${r['costo_inventario']:,.0f}</div>
      </div>
    """

# ========= Figuras base (si no están creadas, se crean rápido) =========
def to_html_fig(fig):
    return pio.to_html(fig, include_plotlyjs=False, full_html=False) if fig is not None else ""

# Días por tienda
if 'fig_dias_tienda' in globals():
    fig_dias_tienda_html = to_html_fig(fig_dias_tienda)
else:
    dt = dias_tienda.copy()
    if 'semaforo' not in dt.columns:
        def _sem(d):
            if d<=26: return 'Verde (≤26)'
            if d<=45: return 'Amarillo (27–45)'
            return 'Rojo (>45)'
        dt['dias_pesos'] = dt['dias_pesos'].replace([np.inf,-np.inf], np.nan).fillna(0)
        dt['semaforo'] = dt['dias_pesos'].apply(_sem)
    fig_dt = px.bar(
        dt.sort_values('dias_pesos', ascending=False),
        x='nombre_tienda', y='dias_pesos', color='semaforo',
        color_discrete_map={'Verde (≤26)':GREEN_MIL,'Amarillo (27–45)':AMARILLO,'Rojo (>45)':ROJO},
        title=f"Días de Inventario por Tienda (EN PESOS · ventas*{FACTOR_COSTO:.2f}/{DIAS_OPERADOS})",
        text='dias_pesos'
    )
    fig_dt.update_traces(texttemplate='%{text:.1f}', textposition='outside', cliponaxis=False)
    fig_dt.update_layout(yaxis_title='Días', xaxis_title='Tienda', margin=dict(l=40,r=40,t=70,b=80))
    fig_dias_tienda_html = to_html_fig(fig_dt)

# Pareto global en pesos
if 'fig_pareto_global' in globals():
    fig_pareto_global_html = to_html_fig(fig_pareto_global)
else:
    if 'pg' not in globals():
        raise ValueError("No encuentro 'pg' (Pareto global). Ejecuta las celdas de Pareto antes.")
    fg = px.bar(
        pg.sort_values('costo_inventario', ascending=False),
        x='descripcion', y='costo_inventario',
        title="Pareto Global en Pesos (20% del costo)", text='costo_inventario',
        color_discrete_sequence=[COMEX_BLUE]
    )
    fg.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)
    fg.update_layout(yaxis_title='Costo inventario', xaxis_title='SKU / Descripción', margin=dict(l=40,r=40,t=70,b=120))
    fig_pareto_global_html = to_html_fig(fg)

# Pareto global en días
if 'dias_pareto_global_html' in globals():
    dias_pareto_global_html = globals()['dias_pareto_global_html']
else:
    tmp = pg.copy()
    tmp['dias_pesos'] = tmp['dias_pesos'].replace([np.inf,-np.inf], np.nan).fillna(0)
    fd = px.bar(
        tmp.sort_values('dias_pesos', ascending=False),
        x='descripcion', y='dias_pesos',
        title="Pareto Global en Días (20% del costo)", text='dias_pesos',
        color_discrete_sequence=[AMARILLO]
    )
    fd.update_traces(texttemplate='%{text:.1f}', textposition='outside', cliponaxis=False)
    fd.update_layout(yaxis_title='Días (en pesos)', xaxis_title='SKU / Descripción', margin=dict(l=40,r=40,t=70,b=120))
    dias_pareto_global_html = to_html_fig(fd)

# Pastel inventario por tienda
if 'pie_html' in globals():
    pie_html = globals()['pie_html']
else:
    inv_tienda_val = dias_tienda[['nombre_tienda','costo_inventario']].copy()
    fp = px.pie(inv_tienda_val, names='nombre_tienda', values='costo_inventario',
                title="Distribución del Costo de Inventario por Tienda", hole=0.35)
    fp.update_traces(textposition='inside', textinfo='percent+label')
    pie_html = to_html_fig(fp)

# Pareto por tienda (pesos)
if 'pareto_tienda_html' not in globals():
    pth = ""
    for t, dft in pareto_tienda_dict.items():
        if dft is None or len(dft)==0: continue
        d = dft.sort_values('costo_inventario', ascending=False).copy()
        ft = px.bar(d, x='descripcion', y='costo_inventario',
                    title=f"Pareto por Tienda en Pesos — {t}", text='costo_inventario',
                    color_discrete_sequence=[GREEN_MIL])
        ft.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)
        ft.update_layout(yaxis_title='Costo inventario', xaxis_title='SKU / Descripción', margin=dict(l=40,r=40,t=60,b=120))
        pth += to_html_fig(ft)
    pareto_tienda_html = pth
else:
    pareto_tienda_html = globals()['pareto_tienda_html']

# Pareto por tienda (días)
if 'pareto_dias_tienda_html' not in globals():
    pdt = ""
    for t, dft in pareto_tienda_dict.items():
        if dft is None or len(dft)==0: continue
        d = dft.copy()
        if 'dias_pesos' not in d.columns:
            d['costo_diario'] = (d['subtotal'] * FACTOR_COSTO) / max(DIAS_OPERADOS,1)
            d['dias_pesos']   = np.where(d['costo_diario']>0, d['costo_inventario']/d['costo_diario'], np.inf)
        d['dias_pesos'] = d['dias_pesos'].replace([np.inf,-np.inf], np.nan).fillna(0)
        ft = px.bar(d.sort_values('dias_pesos', ascending=False), x='descripcion', y='dias_pesos',
                    title=f"Pareto por Tienda en Días — {t}", text='dias_pesos',
                    color_discrete_sequence=[AMARILLO])
        ft.update_traces(texttemplate='%{text:.1f}', textposition='outside', cliponaxis=False)
        ft.update_layout(yaxis_title='Días (en pesos)', xaxis_title='SKU / Descripción', margin=dict(l=40,r=40,t=60,b=120))
        pdt += to_html_fig(ft)
    pareto_dias_tienda_html = pdt
else:
    pareto_dias_tienda_html = globals()['pareto_dias_tienda_html']

# Ahorros 30/26
if 'ahorro_30_html' in globals() and 'ahorro_26_html' in globals():
    ahorro_30_html = globals()['ahorro_30_html']
    ahorro_26_html = globals()['ahorro_26_html']
else:
    dt2 = dias_tienda.copy()
    if 'ahorro_30' not in dt2.columns: dt2['ahorro_30'] = np.maximum(0.0, dt2['costo_inventario'] - dt2['costo_diario']*30)
    if 'ahorro_26' not in dt2.columns: dt2['ahorro_26'] = np.maximum(0.0, dt2['costo_inventario'] - dt2['costo_diario']*26)
    f30 = px.bar(dt2.sort_values('ahorro_30', ascending=False), x='nombre_tienda', y='ahorro_30',
                 title="Ahorro Potencial por Tienda (30 días)", text='ahorro_30', color_discrete_sequence=[GREEN_MIL])
    f30.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)
    f26 = px.bar(dt2.sort_values('ahorro_26', ascending=False), x='nombre_tienda', y='ahorro_26',
                 title="Ahorro Potencial por Tienda (26 días)", text='ahorro_26', color_discrete_sequence=[COMEX_BLUE])
    f26.update_traces(texttemplate='$%{text:,.0f}', textposition='outside', cliponaxis=False)
    ahorro_30_html = to_html_fig(f30)
    ahorro_26_html = to_html_fig(f26)

# ========= Gráfica de burbuja: Portafolio por tienda =========
dtp = dias_tienda.copy()
if 'semaforo' not in dtp.columns:
    def _sem(d):
        if d<=26: return 'Verde (≤26)'
        if d<=45: return 'Amarillo (27–45)'
        return 'Rojo (>45)'
    dtp['semaforo'] = dtp['dias_pesos'].apply(_sem)

dtp['dias_pesos'] = dtp['dias_pesos'].replace([np.inf, -np.inf], np.nan).fillna(0)
dtp['venta_diaria'] = dtp['costo_diario']  # costo diario ya calculado
dtp['size_bubble']  = np.sqrt(dtp['costo_inventario'].clip(lower=0))

fig_portafolio_tiendas = px.scatter(
    dtp, x='venta_diaria', y='dias_pesos', size='size_bubble', color='semaforo',
    hover_data={'nombre_tienda':True,'costo_inventario':':,.0f','venta_diaria':':,.0f','dias_pesos':':.1f','size_bubble':False},
    color_discrete_map={'Verde (≤26)':GREEN_MIL,'Amarillo (27–45)':AMARILLO,'Rojo (>45)':ROJO},
    title="Portafolio por Tienda — Venta diaria vs Días (tamaño = costo inventario)"
)
fig_portafolio_tiendas.update_layout(
    xaxis_title="Venta diaria (costo) = ventas*{:.0%}/{}".format(FACTOR_COSTO, DIAS_OPERADOS),
    yaxis_title="Días de inventario (en pesos)",
    margin=dict(l=40,r=40,t=70,b=60)
)
portafolio_tiendas_html = to_html_fig(fig_portafolio_tiendas)

# ========= Estilos =========
style = f"""
<style>
body {{ font-family: Inter, Arial, sans-serif; background: {BG_LIGHT}; margin: 0; }}
.header {{ display:flex; align-items:center; justify-content:center; gap:16px; padding:24px 28px; background:{CARD_BG}; border-bottom:1px solid {BORDER}; }}
.header h1 {{ margin:0; color:{GRIS_TXT}; font-size:32px; font-weight:800; }}

.section {{ padding: 28px; }}
.block {{ background:{CARD_BG}; border:1px solid {BORDER}; border-radius:14px; padding:16px; }}
.section h2 {{ color:{GRIS_TXT}; font-size:22px; margin:8px 0 16px 0; text-align:center; }}

.kpi-wrap {{ display:flex; gap:16px; justify-content:center; flex-wrap:wrap; }}
.card {{ background:{CARD_BG}; border:1px solid {BORDER}; border-radius:14px; padding:18px 20px; min-width:260px; box-shadow:0 2px 10px rgba(0,0,0,0.04); text-align:center; }}
.kpi-title {{ font-size:13px; color:#6b7280; margin-bottom:6px; text-transform:uppercase; letter-spacing:.04em; }}
.kpi-value {{ font-size:24px; font-weight:800; color:{GRIS_TXT}; }}
.kpi-sub {{ font-size:12px; color:#7f8c8d; margin-top:4px; }}

.chips {{ display:flex; gap:10px; row-gap:10px; flex-wrap:wrap; justify-content:center; }}
.chip {{
  background:{CARD_BG}; border:1px solid {BORDER}; border-radius:999px;
  padding:8px 14px; display:flex; align-items:center; gap:10px;
  box-shadow:0 1px 6px rgba(0,0,0,0.05);
}}
.chip-name {{ font-size:13px; color:#374151; font-weight:600; }}
.chip-value {{ font-size:13px; color:{COMEX_BLUE}; font-weight:800; }}
</style>
"""

# ========= HTML completo =========
html_out = f"""
<!doctype html>
<html lang="es">
<head>
<meta charset="utf-8">
{style}
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
<title>Dashboard de Optimización de Inventario</title>
</head>
<body>

<div class="header"><h1>Dashboard de Optimización de Inventario</h1></div>

<div class="section">
  <div class="block"><div class='kpi-wrap'>
    {kpi_html}
  </div></div>
</div>

<div class="section">
  <h2>Inventario en Pesos por Tienda</h2>
  <div class="block">
    <div class="chips">
      {chips_html}
    </div>
  </div>
</div>

<div class="section">
  <h2>Días de Inventario por Tienda</h2>
  <div class="block">{fig_dias_tienda_html}</div>
</div>

<div class="section">
  <h2>Pareto Global en Pesos (20% del costo)</h2>
  <div class="block">{fig_pareto_global_html}</div>
</div>

<div class="section">
  <h2>Pareto Global en Días</h2>
  <div class="block">{dias_pareto_global_html}</div>
</div>

<div class="section">
  <h2>Distribución del Costo de Inventario por Tienda</h2>
  <div class="block">{pie_html}</div>
</div>

<div class="section">
  <h2>Pareto por Tienda en Pesos (20% del costo)</h2>
  <div class="block">{pareto_tienda_html if pareto_tienda_html else "<div style='padding:10px;color:#6b7280;'>Sin datos para mostrar.</div>"}</div>
</div>

<div class="section">
  <h2>Pareto por Tienda en Días</h2>
  <div class="block">{pareto_dias_tienda_html if pareto_dias_tienda_html else "<div style='padding:10px;color:#6b7280;'>Sin datos para mostrar.</div>"}</div>
</div>

<div class="section">
  <h2>Portafolio por Tienda — Venta diaria vs Días</h2>
  <div class="block">{portafolio_tiendas_html}</div>
</div>

<div class="section">
  <h2>Ahorro Potencial por Tienda (30 días)</h2>
  <div class="block">{ahorro_30_html}</div>
</div>

<div class="section">
  <h2>Ahorro Potencial por Tienda (26 días)</h2>
  <div class="block">{ahorro_26_html}</div>
</div>

</body>
</html>
"""

with open("Dashboard_Inventario.html","w",encoding="utf-8") as f:
    f.write(html_out)

print("✅ Dashboard generado con chips de inventario y burbuja: Dashboard_Inventario.html")


✅ Dashboard generado con chips de inventario y burbuja: Dashboard_Inventario.html


In [25]:
import plotly.io as pio
import numpy as np

# Colores/estilos por si no existen
BG_LIGHT = globals().get("BG_LIGHT", "#F3F7FA")
CARD_BG  = globals().get("CARD_BG",  "#FFFFFF")
BORDER   = globals().get("BORDER",   "#E5E7EB")
GRIS_TXT = globals().get("GRIS_TXT", "#2C3E50")

# KPI HTML que ya usabas arriba (si no existe, deja uno mínimo)
if "kpi_html" not in globals():
    kpi_html = """
    <div class="card"><div class="kpi-title">Ventas 90d</div><div class="kpi-value">$0</div></div>
    <div class="card"><div class="kpi-title">Costo Inventario</div><div class="kpi-value">$0</div></div>
    """

# Chips de inventario por tienda (si no los tienes ya construidos)
chips_html = globals().get("chips_html", "")

# Asegura embebidos (si tus figuras están como objetos fig_*, convértelas)
def _ensure_html(var_name, fig_obj_name=None):
    if var_name in globals() and isinstance(globals()[var_name], str) and globals()[var_name]:
        return globals()[var_name]
    fig = globals().get(fig_obj_name) if fig_obj_name else None
    return pio.to_html(fig, include_plotlyjs=False, full_html=False) if fig is not None else ""

fig_dias_tienda_html   = _ensure_html("fig_dias_tienda_html",   "fig_dias_tienda")
fig_pareto_global_html = _ensure_html("fig_pareto_global_html", "fig_pareto_global")
dias_pareto_global_html = globals().get("dias_pareto_global_html", "")
pareto_tienda_html      = globals().get("pareto_tienda_html", "")
pareto_dias_tienda_html = globals().get("pareto_dias_tienda_html", "")
pie_html                = _ensure_html("pie_html", "fig_pie_inv")
ahorro_30_html          = globals().get("ahorro_30_html", "")
ahorro_26_html          = globals().get("ahorro_26_html", "")

style = f"""
<style>
body {{ font-family: Inter, Arial, sans-serif; background: {BG_LIGHT}; margin: 0; }}
.header {{ display:flex; align-items:center; justify-content:center; gap:16px; padding:24px 28px; background:{CARD_BG}; border-bottom:1px solid {BORDER}; }}
.header h1 {{ margin:0; color:{GRIS_TXT}; font-size:32px; font-weight:800; }}

.section {{ padding: 28px; }}
.block {{ background:{CARD_BG}; border:1px solid {BORDER}; border-radius:14px; padding:16px; }}
.section h2 {{ color:{GRIS_TXT}; font-size:22px; margin:8px 0 16px 0; text-align:center; }}

.kpi-wrap {{ display:flex; gap:16px; justify-content:center; flex-wrap:wrap; }}
.card {{ background:{CARD_BG}; border:1px solid {BORDER}; border-radius:14px; padding:18px 20px; min-width:260px; box-shadow:0 2px 10px rgba(0,0,0,0.04); text-align:center; }}
.kpi-title {{ font-size:13px; color:#6b7280; margin-bottom:6px; text-transform:uppercase; letter-spacing:.04em; }}
.kpi-value {{ font-size:24px; font-weight:800; color:{GRIS_TXT}; }}
.kpi-sub {{ font-size:12px; color:#7f8c8d; margin-top:4px; }}

.chips {{ display:flex; gap:10px; row-gap:10px; flex-wrap:wrap; justify-content:center; }}
.chip {{
  background:{CARD_BG}; border:1px solid {BORDER}; border-radius:999px;
  padding:8px 14px; display:flex; align-items:center; gap:10px;
  box-shadow:0 1px 6px rgba(0,0,0,0.05);
}}
.chip-name {{ font-size:13px; color:#374151; font-weight:600; }}
.chip-value {{ font-size:13px; color:#00ADEF; font-weight:800; }}

.search-wrap {{ display:flex; justify-content:center; }}
.search-box {{
  width: min(680px, 92%);
  display:flex; gap:8px; align-items:center; margin: 6px auto 16px auto;
}}
.search-box input {{
  flex:1; padding:10px 12px; border:1px solid {BORDER}; border-radius:10px; font-size:14px;
}}
.search-box small {{ color:#6b7280; }}
</style>
"""

# Script JS: buscador general que filtra TODAS las gráficas dentro de contenedores .pareto-container
# Funciona para barras (bar charts) de Plotly. Guarda original en gd._origData la 1a vez.
script = """
<script>
function _debounce(fn, ms){ let t; return (...args)=>{ clearTimeout(t); t=setTimeout(()=>fn.apply(this,args),ms); }; }

function initParetoSnapshots(){
  const blocks = document.querySelectorAll('.pareto-container .plotly-graph-div');
  blocks.forEach((gd)=>{
    if (!gd._origData){
      // snapshot de datos originales de todas las trazas
      gd._origData = gd.data.map(tr=>({
        x: (tr.x||[]).slice(),
        y: (tr.y||[]).slice(),
        text: tr.text ? (Array.isArray(tr.text) ? tr.text.slice() : tr.text) : null,
        marker: tr.marker ? JSON.parse(JSON.stringify(tr.marker)) : null,
        name: tr.name || null,
        type: tr.type || 'bar',
        orientation: tr.orientation || undefined
      }));
    }
  });
}

function filterParetoCharts(q){
  const term = (q||'').trim().toLowerCase();
  const blocks = document.querySelectorAll('.pareto-container .plotly-graph-div');
  blocks.forEach((gd)=>{
    if (!gd._origData){ return; }
    const newData = gd._origData.map((tr)=>{
      // Solo filtramos barras con eje X categórico
      const x = tr.x || [];
      const y = tr.y || [];
      let textArr = Array.isArray(tr.text) ? tr.text : null;

      if (!term){
        return {
          x: x, y: y,
          text: textArr || tr.text || null,
          type: tr.type || 'bar',
          name: tr.name || null,
          marker: tr.marker || undefined,
          orientation: tr.orientation || undefined
        };
      }

      // filtro por coincidencia en etiqueta X
      const xf=[], yf=[], tf=[];
      for (let i=0;i<x.length;i++){
        const label = (x[i]===null || x[i]===undefined) ? '' : String(x[i]).toLowerCase();
        if (label.indexOf(term) !== -1){
          xf.push(x[i]);
          yf.push(y[i]);
          if (textArr){ tf.push(textArr[i]); }
        }
      }
      return {
        x: xf, y: yf,
        text: textArr ? tf : (tr.text || null),
        type: tr.type || 'bar',
        name: tr.name || null,
        marker: tr.marker || undefined,
        orientation: tr.orientation || undefined
      };
    });
    Plotly.react(gd, newData, gd.layout);
  });
}

const debouncedFilter = _debounce((ev)=>{
  const q = ev.target.value;
  filterParetoCharts(q);
}, 220);

document.addEventListener('DOMContentLoaded', ()=>{
  initParetoSnapshots();
  const input = document.getElementById('productSearch');
  if (input){
    input.addEventListener('keyup', debouncedFilter);
    input.addEventListener('change', debouncedFilter);
  }
});
</script>
"""

html_out = f"""
<!doctype html>
<html lang="es">
<head>
<meta charset="utf-8">
{style}
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
<title>Dashboard de Optimización de Inventario</title>
</head>
<body>

<div class="header"><h1>Dashboard de Optimización de Inventario</h1></div>

<div class="section">
  <div class="block"><div class='kpi-wrap'>
    {kpi_html}
  </div></div>
</div>

<div class="section">
  <h2>Inventario en Pesos por Tienda</h2>
  <div class="block">
    <div class="chips">
      {chips_html}
    </div>
  </div>
</div>

<div class="section">
  <h2>Buscador de Productos (aplica a TODOS los Paretos)</h2>
  <div class="block">
    <div class="search-wrap">
      <div class="search-box">
        <input id="productSearch" type="text" placeholder="Escribe parte de la descripción o SKU para filtrar en los paretos..." />
      </div>
    </div>
    <div style="text-align:center;"><small>Filtra las barras de los paretos (global y por tienda) por coincidencia en el eje X.</small></div>
  </div>
</div>

<div class="section">
  <h2>Días de Inventario por Tienda</h2>
  <div class="block">{fig_dias_tienda_html}</div>
</div>

<div class="section pareto-container">
  <h2>Pareto Global en Pesos (20% del costo)</h2>
  <div class="block">{fig_pareto_global_html}</div>
</div>

<div class="section pareto-container">
  <h2>Pareto Global en Días</h2>
  <div class="block">{dias_pareto_global_html}</div>
</div>

<div class="section">
  <h2>Distribución del Costo de Inventario por Tienda</h2>
  <div class="block">{pie_html}</div>
</div>

<div class="section pareto-container">
  <h2>Pareto por Tienda en Pesos (20% del costo)</h2>
  <div class="block">{pareto_tienda_html if pareto_tienda_html else "<div style='padding:10px;color:#6b7280;'>Sin datos para mostrar.</div>"}</div>
</div>

<div class="section pareto-container">
  <h2>Pareto por Tienda en Días</h2>
  <div class="block">{pareto_dias_tienda_html if pareto_dias_tienda_html else "<div style='padding:10px;color:#6b7280;'>Sin datos para mostrar.</div>"}</div>
</div>

<div class="section">
  <h2>Portafolio por Tienda — Venta diaria vs Días</h2>
  <div class="block">{globals().get('portafolio_tiendas_html', '')}</div>
</div>

<div class="section">
  <h2>Ahorro Potencial por Tienda (30 días)</h2>
  <div class="block">{ahorro_30_html}</div>
</div>

<div class="section">
  <h2>Ahorro Potencial por Tienda (26 días)</h2>
  <div class="block">{ahorro_26_html}</div>
</div>

{script}
</body>
</html>
"""

with open("Dashboard_Inventario.html","w",encoding="utf-8") as f:
    f.write(html_out)

print("✅ Dashboard con BUSCADOR listo: Dashboard_Inventario.html")


✅ Dashboard con BUSCADOR listo: Dashboard_Inventario.html
